In [ ]:
#import packages
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler

In [ ]:
# Load datasets and feature select
hour_df = pd.read_csv('/content/hour.csv')
day_df = pd.read_csv('/content/day.csv')

# Merge of dataset
merged_df = pd.merge(hour_df, day_df, on='dteday', how='left')
merged_df.head()

,instant_x,dteday,season_x,yr_x,mnth_x,hr,holiday_x,weekday_x,workingday_x,weathersit_x,...,weekday_y,workingday_y,weathersit_y,temp_y,atemp_y,hum_y,windspeed_y,casual_y,registered_y,cnt_y
0,1,2011-01-01,1,0,1,0,0,6,0,1,...,6,0,2,0.344167,0.363625,0.805833,0.160446,331,654,985
1,2,2011-01-01,1,0,1,1,0,6,0,1,...,6,0,2,0.344167,0.363625,0.805833,0.160446,331,654,985
2,3,2011-01-01,1,0,1,2,0,6,0,1,...,6,0,2,0.344167,0.363625,0.805833,0.160446,331,654,985
3,4,2011-01-01,1,0,1,3,0,6,0,1,...,6,0,2,0.344167,0.363625,0.805833,0.160446,331,654,985
4,5,2011-01-01,1,0,1,4,0,6,0,1,...,6,0,2,0.344167,0.363625,0.805833,0.160446,331,654,985


In [ ]:
#all column names after merge
print(merged_df.columns.tolist())

['instant_x', 'dteday', 'season_x', 'yr_x', 'mnth_x', 'hr', 'holiday_x', 'weekday_x', 'workingday_x', 'weathersit_x', 'temp_x', 'atemp_x', 'hum_x', 'windspeed_x', 'casual_x', 'registered_x', 'cnt_x', 'instant_y', 'season_y', 'yr_y', 'mnth_y', 'holiday_y', 'weekday_y', 'workingday_y', 'weathersit_y', 'temp_y', 'atemp_y', 'hum_y', 'windspeed_y', 'casual_y', 'registered_y', 'cnt_y']


In [ ]:
# Hour dataset features
features = ['temp_x', 'hum_x', 'windspeed_x', 'hr', 'weekday_y', 'workingday_y']

# Target column
target = 'cnt_x'

# Feature matrix and target vector
X = merged_df[features].values.astype(np.float64)
y = merged_df[target].values.astype(np.float64).reshape(-1, 1)

# Add bias column
X = np.hstack((np.ones((X.shape[0], 1)), X))

# Standardize features
scaler = StandardScaler()
X[:, 1:] = scaler.fit_transform(X[:, 1:])

print("X shape:", X.shape)
print("y shape:", y.shape)

X shape: (17379, 7)
y shape: (17379, 1)


In [ ]:
#Compute ridge gradient
def compute_ridge_gradient(X, y, theta, lambda_):
    m = X.shape[0]
    gradient = (2/m) * (np.matmul(X.T, np.matmul(X, theta) - y)) + 2 * lambda_ * theta
    gradient[0] -= 2 * lambda_ * theta[0]
    return gradient

#calculate mse
def compute_mse(y_true, y_pred):
    return np.mean((y_true - y_pred)**2)

In [ ]:
#mini_batch_gradient_descent Function
def mini_batch_gradient_descent(X, y, theta, learning_rate=0.01, n_epochs=100, batch_size=32, lambda_=0.1):
    m = X.shape[0]
    train_losses = []

    for epoch in range(n_epochs):
        # Shuffle data for each epoch
        perm = np.random.permutation(m)
        X_shuffled = X[perm, :]
        y_shuffled = y[perm, :]

        # Iterate over batches
        for start in range(0, m, batch_size):
            end = min(start + batch_size, m)
            X_batch = X_shuffled[start:end, :]
            y_batch = y_shuffled[start:end, :]

            # Gradient update
            gradient = compute_ridge_gradient(X_batch, y_batch, theta, lambda_)
            theta = theta - learning_rate * gradient

        # Track training loss
        y_pred = np.matmul(X, theta)
        train_losses.append(np.mean((y - y_pred)**2))

    return theta, train_losses

In [ ]:
#hyperparameters
learning_rate = 0.001
n_epochs = 100
batch_size = 50
lambda_ = 0.1

In [ ]:
#K-fold Cross Validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
mse_list = []

for fold, (train_index, test_index) in enumerate(kf.split(X), 1):
    X_train = X[train_index, :].copy()
    X_test  = X[test_index, :].copy()
    y_train = y[train_index, :].copy()
    y_test  = y[test_index, :].copy()

    # Random initialization of theta per fold
    theta_init = np.random.randn(X.shape[1], 1)

    # Train Ridge Regression using MBGD
    theta_trained, train_losses = mini_batch_gradient_descent(X_train, y_train, theta_init,learning_rate, n_epochs, batch_size, lambda_)

    # Evaluate on test set
    y_pred = np.matmul(X_test, theta_trained)
    mse = compute_mse(y_test, y_pred)
    mse_list.append(mse)

    print(f"Fold {fold} MSE {mse:f}")

print(f"Mean MSE {np.mean(mse_list):f}")

Fold 1 MSE 21064.112074
Fold 2 MSE 21797.995752
Fold 3 MSE 22405.552304
Fold 4 MSE 23014.178212
Fold 5 MSE 23043.680187
Mean MSE 22265.103706
